In [1]:
import sys, math, random, gc

import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torch.autograd as autograd 
import torch.nn.functional as F
import torch.cuda as cuda

In [2]:
from IPython.display import clear_output
import matplotlib.pyplot as plt
%matplotlib inline

# Use Cuda

In [3]:
USE_CUDA = torch.cuda.is_available()
Variable = lambda *args, **kwargs: autograd.Variable(*args, **kwargs).cuda() if USE_CUDA else autograd.Variable(*args, **kwargs)
print(USE_CUDA)

True


# Judger

In [4]:
ini_map = np.array([[0, 0, 10, 1, 5, 1, 20, 0, 0],
                  [0, 0, 0, 1, 2, 1, 0, 0, 0],
                  [0, 0, 0, 0, 1, 0, 0, 0, 0],
                  [0, 0, 0, 0, 1, 0, 0, 0, 0],
                  [1, 1, 1, 1, 1, 1, 1, 1, 1],
                  [0, 0, 0, 0, 1, 0, 0, 0, 0],
                  [0, 0, 0, 0, 1, 0, 0, 0, 0],
                  [0, 0, 0, 1, 2, 1, 0, 0, 0],
                  [0, 0, -20, 1, 5, 1, -10, 0, 0]])
ini_map_without_tank = np.array([[0, 0, 0, 1, 5, 1, 0, 0, 0],
                  [0, 0, 0, 1, 2, 1, 0, 0, 0],
                  [0, 0, 0, 0, 1, 0, 0, 0, 0],
                  [0, 0, 0, 0, 1, 0, 0, 0, 0],
                  [1, 1, 1, 1, 1, 1, 1, 1, 1],
                  [0, 0, 0, 0, 1, 0, 0, 0, 0],
                  [0, 0, 0, 0, 1, 0, 0, 0, 0],
                  [0, 0, 0, 1, 2, 1, 0, 0, 0],
                  [0, 0, 0, 1, 5, 1, 0, 0, 0]])
ini_tank = np.array([[0, 0, 10, 0, 0, 0, 20, 0, 0],
                   [0, 0, 0, 0, 0, 0, 0, 0, 0],
                   [0, 0, 0, 0, 0, 0, 0, 0, 0],
                   [0, 0, 0, 0, 0, 0, 0, 0, 0],
                   [0, 0, 0, 0, 0, 0, 0, 0, 0],
                   [0, 0, 0, 0, 0, 0, 0, 0, 0],
                   [0, 0, 0, 0, 0, 0, 0, 0, 0],
                   [0, 0, 0, 0, 0, 0, 0, 0, 0],
                   [0, 0, -20, 0, 0, 0, -10, 0, 0]])
ini_tankposx = np.array([[0, 0], [8, 8]])
ini_tankposy = np.array([[2, 6], [6, 2]])
delta = np.array([[-1, 0], [0, -1], [-1, 0], [0, -1], [], [0, 1], [1, 0], [0, 1], [1, 0]])
tank_transform = [0, 0, 0, 0, -10, -20, 10, 20]

class Environment(object):
    def __init__(self):
        #空：0，砖：1，钢：2，基地：3, 甲方坦克：4，5，乙方坦克：6，7
        self.state = ini_map.copy()
        self.tank_posx = ini_tankposx.copy()
        self.tank_posy = ini_tankposy.copy()
        self.if_can_shoot = np.array([[1, 1], [1, 1]])
        self.state_sum = np.array([ini_map_without_tank, ini_tank, np.ones((9, 9), dtype = 'int32'), np.ones((9, 9), dtype = 'int32'), np.ones((9, 9), dtype = 'int32'), np.ones((9, 9), dtype = 'int32')])
    def reset(self, rd_threshold):
        self.state = ini_map.copy()
        self.tank_posx = ini_tankposx.copy()
        self.tank_posy = ini_tankposy.copy()
        self.if_can_shoot = np.array([[1, 1], [1, 1]])
        self.state_sum = np.array([ini_map_without_tank, ini_tank, np.ones((9, 9), dtype = 'int32'), np.ones((9, 9), dtype = 'int32'), np.ones((9, 9), dtype = 'int32'), np.ones((9, 9), dtype = 'int32')])
        for i in range(9):
            for j in range(9):
                if self.state[i][j] == 0:
                    self.state[i][j] = (np.random.random() > rd_threshold)
                    self.state_sum[0][i][j] = self.state[i][j]
        #state_temp = self.state.copy()
        #state_temp[np.reshape(self.tank_posx, (-1)), np.reshape(self.tank_posy, (-1))] = [-10, -20, 10, 20]
        #print(self.state)
        #return [self.state.copy(), np.flip(np.flip(state_temp, 0), 1)]
    def InMap(self, pos):
        if (pos[0] >= 0 and pos[0] < 9 and pos[1] >= 0 and pos[1] < 9):
            return True
        return False
    def step(self, action):
        """action编号：
        0：上移
        1：左移
        2：上射
        3：左射
        4：静止
        5：右射
        6：下射
        7：右移
        8：下移
        """
        #Player0
        action00 = int(action[0] / 9)
        action01 = action[0] % 9
        reward0 = 0
        done0 = 0
        nextpos00 = np.array([self.tank_posx[0][0], self.tank_posy[0][0]])
        nextpos01 = np.array([self.tank_posx[0][1], self.tank_posy[0][1]])
        obj_shot00 = np.array([-1, -1])
        obj_shot01 = np.array([-1, -1])
        
        #Tank0
        if (nextpos00[0] != -1 and action00 != 4): #tank isn't dead
            if (action00 < 2 or action00 > 6): #move
                nextpos00 += delta[action00]
        
        #Tank1
        if (nextpos01[0] != -1 and action01 != 4):
            if (action01 < 2 or action01 > 6): #move
                nextpos01 += delta[action01]
         
        #Player1
        action10 = 8 - int(action[1] / 9)
        action11 = 8 - action[1] % 9
        reward1 = 0
        done1 = 0
        nextpos10 = np.array([self.tank_posx[1][0], self.tank_posy[1][0]])
        nextpos11 = np.array([self.tank_posx[1][1], self.tank_posy[1][1]])
        obj_shot10 = np.array([-1, -1])
        obj_shot11 = np.array([-1, -1])
        
        #Tank0
        if (nextpos10[0] != -1 and action10 != 4): #tank isn't dead
            if (action10 < 2 or action10 > 6): #move
                nextpos10 += delta[action10]
                
        
        #Tank1
        if (nextpos11[0] != -1 and action11 != 4):
            if (action11 < 2 or action11 > 6): #move
                nextpos11 += delta[action11]
        
        
        #State Update
        nextpos = [[nextpos00, nextpos01], [nextpos10, nextpos11]]
        for i in range(2):
            for j in range(2):
                if (self.tank_posx[i][j] != -1 and self.InMap(nextpos[i][j])):
                    self.state[self.tank_posx[i][j], self.tank_posy[i][j]], self.state[nextpos[i][j][0], nextpos[i][j][1]] = self.state[nextpos[i][j][0], nextpos[i][j][1]], self.state[self.tank_posx[i][j], self.tank_posy[i][j]] #swap
                    self.tank_posx[i][j], self.tank_posy[i][j] = nextpos[i][j][0], nextpos[i][j][1]
        #print(self.state)
        
        #Shoot
        #Player0 
        #Tank0
        if (nextpos00[0] != -1 and action00 != 4): #not dead
            if (action00 > 1 and action00 < 7): #shoot
                temp_pos = nextpos00 + delta[action00]
                while (self.InMap(temp_pos)):
                    #print(temp_pos)
                    if (self.state[temp_pos[0], temp_pos[1]] != 0):
                        obj_shot00 = temp_pos.copy()
                        break
                    temp_pos += delta[action00]
        
        #Tank1
        if (nextpos01[0] != -1 and action01 != 4):
            if (action01 > 1 and action01 < 7): #shoot
                temp_pos = nextpos01 + delta[action01]
                while (self.InMap(temp_pos)):
                    if (self.state[temp_pos[0], temp_pos[1]] != 0):
                        obj_shot01 = temp_pos.copy()
                        break
                    temp_pos += delta[action01]
        
        #Player1
        #Tank0
        if (nextpos10[0] != -1 and action10 != 4): #not dead
            if (action10 > 1 and action10 < 7): #shoot
                temp_pos = nextpos10 + delta[action10]
                while (self.InMap(temp_pos)):
                    if (self.state[temp_pos[0], temp_pos[1]] != 0):
                        obj_shot10 = temp_pos.copy()
                        #print(obj_shot10)
                        break
                    temp_pos += delta[action10]

        #Tank1
        if (nextpos11[0] != -1 and action11 != 4): #not dead
            if (action11 > 1 and action11 < 7): #shoot
                temp_pos = nextpos11 + delta[action11]
                while (self.InMap(temp_pos)):
                    if (self.state[temp_pos[0], temp_pos[1]] != 0):
                        obj_shot11 = temp_pos.copy()
                        #print(obj_shot11)
                        break
                    temp_pos += delta[action11]
         
        #Anything got shot?
        obj_shot = np.array([[obj_shot00, obj_shot01], [obj_shot10, obj_shot11]])
        for i in range(2):
            for j in range(2):
                if (obj_shot[i][j][0] != -1):
                    if (abs(self.state[obj_shot[i][j][0], obj_shot[i][j][1]]) >= 10): #Tank got shot
                        tanknum = self.state[obj_shot[i][j][0], obj_shot[i][j][1]]
                        if (obj_shot[int(tanknum < 0)][int(abs(tanknum) > 15)][0] != self.tank_posx[i][j] or
                            obj_shot[int(tanknum < 0)][int(abs(tanknum) > 15)][1] != self.tank_posy[i][j]): #没有对射
                            reward0 += 1 * (tanknum < 0) * (i == 0) + -1 * (tanknum > 0)
                            reward1 += 1 * (tanknum > 0) * (i == 1) + -1 * (tanknum < 0)

                            self.state[obj_shot[i][j][0], obj_shot[i][j][1]] = 0
                            self.tank_posx[int(tanknum < 0)][int(abs(tanknum) > 15)] = -1
                            self.tank_posy[int(tanknum < 0)][int(abs(tanknum) > 15)] = -1
                    elif (self.state[obj_shot[i][j][0], obj_shot[i][j][1]] == 5):
                        #Base got shot
                        reward0 += 1 * (obj_shot[i][j][0] == 8) * (i == 0) + -1 * (obj_shot[i][j][0] == 0)
                        reward1 += 1 * (obj_shot[i][j][0] == 0) * (i == 1) + -1 * (obj_shot[i][j][0] == 8)
                        done0 = 1
                    elif (self.state[obj_shot[i][j][0], obj_shot[i][j][1]] != 2):
                        self.state[obj_shot[i][j][0], obj_shot[i][j][1]] = 0
                    
        
        
        if (self.tank_posx[0][0] == -1 and self.tank_posx[0][1] == -1):
            done0 = 1
            #reward0 -= 1
            #reward1 += 1
        if (self.tank_posx[1][0] == -1 and self.tank_posx[1][1] == -1):
            done1 = 1
            #reward0 += 1
            #reward1 -= 1
        
        done = (done0 or done1)
        
        self.state_sum[0] = self.state
        self.state_sum[1] = np.zeros((9, 9))
        for i in range(9):
            for j in range(9):
                if abs(self.state[i][j]) >= 10:
                    self.state_sum[1][i][j] = self.state_sum[0][i][j]
                    self.state_sum[0][i][j] = 0
        self.state_sum[2] = np.zeros((9, 9)) + int(action00 < 2 or action00 > 6 or action00 == 4)
        self.state_sum[3] = np.zeros((9, 9)) + int(action01 < 2 or action01 > 6 or action01 == 4)
        self.state_sum[4] = np.zeros((9, 9)) + int(action10 < 2 or action10 > 6 or action10 == 4)
        self.state_sum[5] = np.zeros((9, 9)) + int(action11 < 2 or action11 > 6 or action11 == 4)
        
        v = 0
        if done:
            v = (reward0 > 0) * 1 + (reward0 < 0) * -1
        
        
        return [v, done]

In [5]:
def InMap(pos):
    if (pos[0] >= 0 and pos[0] < 9 and pos[1] >= 0 and pos[1] < 9):
        return True
    return False

def step_temp(action, state_sum):
    """action编号：
    0：上移
    1：左移
    2：上射
    3：左射
    4：静止
    5：右射
    6：下射
    7：右移
    8：下移
    """
    tank_posx = np.array([[-1, -1], [-1, -1]])
    tank_posy = np.array([[-1, -1], [-1, -1]])
    for i in range(9):
        for j in range(9):
            a = state_sum[1][i][j]
            tank_posx[int(a < 0)][int(abs(a) == 20)] += (i + 1) * (a != 0)
            tank_posy[int(a < 0)][int(abs(a) == 20)] += (j + 1) * (a != 0)
    #print(tank_posx, tank_posy)
    
    state = state_sum[0] + state_sum[1]
    if_can_shoot = np.array([[state_sum[2][0][0], state_sum[3][0][0]], [state_sum[4][0][0], state_sum[5][0][0]]])

    
    #Player0
    action00 = int(action[0] / 9)
    action01 = action[0] % 9
    reward0 = 0
    done0 = 0
    nextpos00 = np.array([tank_posx[0][0], tank_posy[0][0]])
    nextpos01 = np.array([tank_posx[0][1], tank_posy[0][1]])
    obj_shot00 = np.array([-1, -1])
    obj_shot01 = np.array([-1, -1])

    #Tank0
    if (nextpos00[0] != -1 and action00 != 4): #tank isn't dead
        if (action00 < 2 or action00 > 6): #move
            nextpos00 += delta[action00]

    #Tank1
    if (nextpos01[0] != -1 and action01 != 4):
        if (action01 < 2 or action01 > 6): #move
            nextpos01 += delta[action01]

    #Player1
    action10 = 8 - int(action[1] / 9)
    action11 = 8 - action[1] % 9
    reward1 = 0
    done1 = 0
    nextpos10 = np.array([tank_posx[1][0], tank_posy[1][0]])
    nextpos11 = np.array([tank_posx[1][1], tank_posy[1][1]])
    obj_shot10 = np.array([-1, -1])
    obj_shot11 = np.array([-1, -1])

    #Tank0
    if (nextpos10[0] != -1 and action10 != 4): #tank isn't dead
        if (action10 < 2 or action10 > 6): #move
            nextpos10 += delta[action10]

    #Tank1
    if (nextpos11[0] != -1 and action11 != 4):
        if (action11 < 2 or action11 > 6): #move
            nextpos11 += delta[action11]


    #State Update
    nextpos = [[nextpos00, nextpos01], [nextpos10, nextpos11]]
    for i in range(2):
        for j in range(2):
            if (tank_posx[i][j] != -1 and InMap(nextpos[i][j])):
                state[tank_posx[i][j], tank_posy[i][j]], state[nextpos[i][j][0], nextpos[i][j][1]] = state[nextpos[i][j][0], nextpos[i][j][1]], state[tank_posx[i][j], tank_posy[i][j]] #swap
                tank_posx[i][j], tank_posy[i][j] = nextpos[i][j][0], nextpos[i][j][1]

    #Shoot
    #Player0 
    #Tank0
    if (nextpos00[0] != -1 and action00 != 4): #not dead
        if (action00 > 1 and action00 < 7): #shoot
            temp_pos = nextpos00 + delta[action00]
            while (InMap(temp_pos)):
                #print(temp_pos)
                if (state[temp_pos[0], temp_pos[1]] != 0):
                    obj_shot00 = temp_pos.copy()
                    break
                temp_pos += delta[action00]

    #Tank1
    if (nextpos01[0] != -1 and action01 != 4):
        if (action01 > 1 and action01 < 7): #shoot
            temp_pos = nextpos01 + delta[action01]
            while (InMap(temp_pos)):
                if (state[temp_pos[0], temp_pos[1]] != 0):
                    obj_shot01 = temp_pos.copy()
                    break
                temp_pos += delta[action01]

    #Player1
    #Tank0
    if (nextpos10[0] != -1 and action10 != 4): #not dead
        if (action10 > 1 and action10 < 7): #shoot
            temp_pos = nextpos10 + delta[action10]
            while (InMap(temp_pos)):
                if (state[temp_pos[0], temp_pos[1]] != 0):
                    obj_shot10 = temp_pos.copy()
                    #print(obj_shot10)
                    break
                temp_pos += delta[action10]

    #Tank1
    if (nextpos11[0] != -1 and action11 != 4): #not dead
        if (action11 > 1 and action11 < 7): #shoot
            temp_pos = nextpos11 + delta[action11]
            while (InMap(temp_pos)):
                if (state[temp_pos[0], temp_pos[1]] != 0):
                    obj_shot11 = temp_pos.copy()
                    #print(obj_shot11)
                    break
                temp_pos += delta[action11]

    #Anything got shot?
    obj_shot = np.array([[obj_shot00, obj_shot01], [obj_shot10, obj_shot11]])
    for i in range(2):
        for j in range(2):
            if (obj_shot[i][j][0] != -1):
                if (abs(state[obj_shot[i][j][0], obj_shot[i][j][1]]) >= 10): #Tank got shot
                    tanknum = state[obj_shot[i][j][0], obj_shot[i][j][1]]
                    if (obj_shot[int(tanknum < 0)][int(abs(tanknum) > 15)][0] != tank_posx[i][j] or
                        obj_shot[int(tanknum < 0)][int(abs(tanknum) > 15)][1] != tank_posy[i][j]): #没有对射
                        reward0 += 1 * (tanknum < 0) * (i == 0) + -1 * (tanknum > 0)
                        reward1 += 1 * (tanknum > 0) * (i == 1) + -1 * (tanknum < 0)

                        state[obj_shot[i][j][0], obj_shot[i][j][1]] = 0
                        tank_posx[int(tanknum < 0)][int(abs(tanknum) > 15)] = -1
                        tank_posy[int(tanknum < 0)][int(abs(tanknum) > 15)] = -1
                elif (state[obj_shot[i][j][0], obj_shot[i][j][1]] == 5):
                    #Base got shot
                    reward0 += 1 * (obj_shot[i][j][0] == 8) * (i == 0) + -1 * (obj_shot[i][j][0] == 0)
                    reward1 += 1 * (obj_shot[i][j][0] == 0) * (i == 1) + -1 * (obj_shot[i][j][0] == 8)
                    done0 = 1
                elif (state[obj_shot[i][j][0], obj_shot[i][j][1]] != 2):
                    state[obj_shot[i][j][0], obj_shot[i][j][1]] = 0



    if (tank_posx[0][0] == -1 and tank_posx[0][1] == -1):
        done0 = 1
        #reward0 -= 1
        #reward1 += 1
    if (tank_posx[1][0] == -1 and tank_posx[1][1] == -1):
        done1 = 1
        #reward0 += 1
        #reward1 -= 1

    done = (done0 or done1)

    state_sum[0] = state
    state_sum[1] = np.zeros((9, 9))
    for i in range(9):
        for j in range(9):
            if abs(state[i][j]) >= 10:
                state_sum[1][i][j] = state_sum[0][i][j]
                state_sum[0][i][j] = 0
    state_sum[2] = np.zeros((9, 9)) + int(action00 < 2 or action00 > 6 or action00 == 4)
    state_sum[3] = np.zeros((9, 9)) + int(action01 < 2 or action01 > 6 or action01 == 4)
    state_sum[4] = np.zeros((9, 9)) + int(action10 < 2 or action10 > 6 or action10 == 4)
    state_sum[5] = np.zeros((9, 9)) + int(action11 < 2 or action11 > 6 or action11 == 4)
    
    v = [0.0, 0.0]
    if done:
        v = [(reward0 > 0) * 1 + (reward0 < 0) * -1, (reward1 > 0) * 1 + (reward1 < 0) * -1]
    
    #print(state_sum[0] + state_sum[1])

    return [state_sum, v, done]

# Network (Unfixed)

In [6]:
def flatten(x):
    N = x.shape[0] # read in N, C, H, W
    return x.view(N, -1)  # "flatten" the C * H * W values into a single vector per image

class Flatten(nn.Module):
    def forward(self, x):
        return flatten(x)

In [7]:
channel_1 = 64
channel_2 = 32
channel_3 = 16

shoot_num = [2, 3, 5, 6]

class DRN(nn.Module):
    def __init__(self):
        super(DRN, self).__init__()
        
        self.layer0 = nn.Sequential(
        nn.Conv2d(6, channel_1, 3, padding = 1), #场地，坦克，四个if_can_shoot
        nn.BatchNorm2d(channel_1),
        nn.ReLU()
        )
        
        self.layer1 = nn.Sequential(
        nn.Conv2d(channel_1, channel_1, 3, padding = 1),
        nn.BatchNorm2d(channel_1)
        )
        
        self.layer2 = nn.Sequential(
        nn.Conv2d(channel_1, channel_1, 3, padding = 1),
        nn.BatchNorm2d(channel_1)
        )
        
        self.layer3 = nn.Sequential(
        nn.Conv2d(channel_1, channel_1, 3, padding = 1),
        nn.BatchNorm2d(channel_1)
        )
        
        self.layer_policy = nn.Sequential(
        nn.Conv2d(channel_1, 2, 1),
        nn.BatchNorm2d(2),
        nn.ReLU(),
        Flatten(),
        nn.Linear(2 * 9 * 9, 81),
        nn.Sigmoid()
        )
        
        self.layer_value = nn.Sequential(
        nn.Conv2d(channel_1, 1, 1),
        nn.BatchNorm2d(1),
        nn.ReLU(),
        Flatten(),
        nn.Linear(1 * 9 * 9, 1),
        nn.Tanh()
        )
        
    def forward(self, x):
        result = self.layer0(x)
        result = F.relu(result + self.layer1(result))
        result = F.relu(result + self.layer2(result))
        result = F.relu(result + self.layer3(result))
        return (self.layer_policy(result), self.layer_value(result))
    
    '''def act(self, state, epsilon, if_can_shoot0, if_can_shoot1):
        if random.random() > epsilon:
            state_temp   = torch.cuda.FloatTensor(state.copy())#, requires_grad = False)
            q_value = self.forward(state_temp)
            if (not if_can_shoot0):
                for num in shoot_num:
                    for j in range(9):
                        q_value[:, num * 9 + j] = -100
            if (not if_can_shoot1):
                for i in range(9):
                    for num in shoot_num:
                        q_value[:, i * 9 + num] = -100
            action  = torch.argmax(q_value, dim = 1).item()
        else:
            action = random.randrange(81) #number of operations
        return action'''

# MCTS

In [8]:
def Zero_To_One(state):
    state[0] = np.flip(np.flip(state[0], 0), 1)
    state[1] = np.flip(np.flip(state[1], 0), 1) * -1
    state[2], state[3], state[4], state[5] = state[4], state[5], state[2], state[3]
    return state

In [9]:
move_number = [0, 1, 7, 8]
shoot_number = [2, 3, 5, 6]

def CalLegal(state):
    legal_moves = np.array([1 for i in range(81)])
    state_temp = state[0] + state[1]
    for i in range(9):
        for j in range(9):
            if state[1][i][j] == 10:
                for k in move_number:
                    if (not InMap([i + delta[k][0], j + delta[k][1]])) or state_temp[i + delta[k][0]][j + delta[k][1]] != 0:
                        legal_moves[k * 9 : k * 9 + 9] = -10000
            if state[1][i][j] == 20:
                for k in move_number:
                    if (not InMap([i + delta[k][0], j + delta[k][1]])) or state_temp[i + delta[k][0]][j + delta[k][1]] != 0:
                        legal_moves[[(x * 9 + k) for x in range(9)]] = -10000
    if state[2][0][0] == 0:
        for k in shoot_number:
            legal_moves[k * 9 : k * 9 + 9] = -10000
    if state[3][0][0] == 0:
        for k in shoot_number:
            legal_moves[[(x * 9 + k) for x in range(9)]] = -10000
    
    return legal_moves

In [10]:
class Node(object):
    def __init__(self, parent_node, state_temp, number, v = 0, if_done = False, side = 0):
        if side == 0:
            self.state = state_temp.copy()
        else:
            self.state = Zero_To_One(state_temp.copy())
        
        self.legal_moves = CalLegal(self.state)
        
        self.parent = parent_node
        self.children = [None for i in range(81)]
        self.edges_N = [0 for i in range(81)]
        self.N_sum = 1
        self.edges_W = [0.0 for i in range(81)]
        self.edges_Q = [0.0 for i in range(81)]
        self.v = [0.0, 0.0]
        self.edges_P, self.v[0] = model(cuda.FloatTensor(np.expand_dims(self.state, axis = 0)))
        self.edges_P = np.reshape(self.edges_P.tolist(), (-1,))
        self.edges_P = (self.edges_P / np.sum(self.edges_P)) * 0.75 + 0.25 * np.random.dirichlet([0.2 for i in range(81)])
        self.v[0] = self.v[0].item()
        self.v[1] = model(cuda.FloatTensor(np.expand_dims(Zero_To_One(self.state.copy()), axis = 0)))[1].item()
        self.edges_U = self.edges_P / 2.0
        self.done = if_done
        #self.action1 = np.argmax(model(cuda.FloatTensor(np.expand_dims(Zero_To_One(self.state.copy()), axis = 0)))[0].tolist())
        
        self.number = number
        self.if_expanded = False
        self.side = side

        if if_done:
            self.v = v

# 选择， 扩展
def tree_policy(node):

    # 选择是否是叶子节点，
    '''if len(node.children) < MAX_CHOICE:
        node = expand(node)
        return node
    else:
        node = best_child(node)'''
    while (node.if_expanded):
        node = best_child(node)
    node.if_expanded = True
    node = best_child(node) #expand

    return node

def backup(node):
    v = node.v
    parent_node = node.parent
    while parent_node != None:
        number = node.number
        parent_node.edges_N[number] += 1
        parent_node.N_sum += 1
        parent_node.edges_W[number] += v[1]
        parent_node.edges_Q[number] = parent_node.edges_W[number] / parent_node.edges_N[number]
        for i in range(81):
            parent_node.edges_U[i] = parent_node.edges_P[i] / 2.0 * np.sqrt(parent_node.N_sum) / (1 + parent_node.edges_N[i])
        node = parent_node
        parent_node = node.parent
        
        number = node.number
        parent_node.edges_N[number] += 1
        parent_node.N_sum += 1
        parent_node.edges_W[number] += v[0]
        parent_node.edges_Q[number] = parent_node.edges_W[number] / parent_node.edges_N[number]
        for i in range(81):
            parent_node.edges_U[i] = parent_node.edges_P[i] / 2.0 * np.sqrt(parent_node.N_sum) / (1 + parent_node.edges_N[i])
        node = parent_node
        parent_node = node.parent

def best_child(node):
    temp = (np.array(node.edges_Q) + np.array(node.edges_U)) * node.legal_moves
    number = np.argmax(temp)
    if (node.children[number] == None):
        #state_temp, v, if_done = step_temp([number, node.action1], node.state.copy())
        node.children[number] = Node(node, node.state.copy(), number, 0, False, 1)
    node1 = node.children[number]
    temp1 = (np.array(node1.edges_Q) + np.array(node1.edges_U)) * node1.legal_moves
    number1 = np.argmax(temp1)
    if (node1.children[number1] == None):
        state_temp, v, if_done = step_temp([number, number1], node.state.copy())
        node1.children[number1] = Node(node1, state_temp, number1, v, if_done, 0)
    
    best = node1.children[number1]
    #print(number, node.action1)
    #print(node.children[number].state[0] + node.children[number].state[1])
    
    return best

def mcts(node, times):
    for i in range(times):

        leaf_node = tree_policy(node)
        backup(leaf_node)

    pi = np.power(np.array(node.edges_N) * node.legal_moves, 2)
    pi[40] += 0.0001
    pi = pi / float(np.sum(pi))
    nextAction0 = np.random.choice(range(81), p = pi)
    node1 = node.children[nextAction0]
    pi1 = np.power(np.array(node1.edges_N) * node1.legal_moves, 2)
    pi1[40] += 0.0001
    nextAction1 = np.random.choice(range(81), p = (pi1 / float(np.sum(pi1))))

    return [nextAction0, nextAction1], (node.state.copy(), pi)

# Loss (Unfixed)

In [11]:
def compute_loss(list_for_train, z):
    size = len(list_for_train)
    state, pi = list(zip(*list_for_train))
    state = np.stack(state, axis = 0)
    pi = np.concatenate(pi)
    z_array = np.array([z for i in range(size)])
    
    p, v = model(cuda.FloatTensor(state))
    loss = (cuda.FloatTensor(z_array) - v.reshape((-1,))).pow(2).mean() - cuda.FloatTensor(pi).dot((p.reshape((-1,)).log())) / size 
        
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    return loss.item()

In [12]:
def plot(losses):
    clear_output(True)
    plt.figure(figsize=(20,5))
    plt.subplot(132)
    plt.title('loss')
    plt.plot(losses)
    plt.show()

# Train (Unfixed)

In [13]:
def If_Can_Shoot_Next(a):
    if (a > 1 and a < 7 and a != 4):
        return False
    return True

In [29]:
learning_rate = 0.0001
model = torch.load('model4_5000.pkl')
model = model.cuda()

    
optimizer = optim.Adam(model.parameters(), lr = learning_rate, weight_decay = 1e-4)

In [31]:
training_turns = 10000
#losses = []
env = Environment()
times_per_search = 5
flush_memory_every = 10000
#list_for_train_winning = []

gc.collect()
for turn in range(5000 + 1, training_turns + 1):
    print(turn)
    if turn % flush_memory_every == 0:
        gc.collect()
    
    env.reset(np.random.random() / 2 + 0.5)
    root = Node(parent_node = None, state_temp = env.state_sum.copy(), number = 0, v = 0, if_done = False, side = 0)
    done = 0
    list_for_train = []
    rounds = 0
    while (not done) and (rounds < 50):
        #print(rounds)
        nextAction, tuple_for_train = mcts(root, times_per_search)
        list_for_train.append(tuple_for_train)
        z, done = env.step(nextAction)
        root = root.children[nextAction[0]].children[nextAction[1]]
        rounds += 1
        #if root.v[0] < -0.8 or root.v[0] > 0.8:
        #    break
    if not done:
        z = root.v[0]
    print(z, rounds)
    loss = compute_loss(list_for_train, z)
    losses.append(loss)
    if z == 1:
        list_for_train_winning.append(list_for_train)
    
    if turn % 10 == 0 and len(list_for_train_winning):
        loss = compute_loss(list_for_train_winning[np.random.randint(len(list_for_train_winning))], 1)
        losses.append(loss)
    
    
    
    if turn % 100 == 0:
        plot(losses)
        

5001
0.9395514726638794 50
5002
-1 7
5003
0.9449776411056519 50
5004
0.8651854395866394 50
5005
0.9406701326370239 50
5006
0.9249318838119507 50
5007
0.9472963213920593 50
5008
0.932766318321228 50
5009
0.9375694394111633 50
5010
0.9413915276527405 50
5011
0.9439395666122437 50
5012
0 31
5013
0.9311802387237549 50
5014
0.9373102188110352 50
5015
0.9348511099815369 50
5016
0.9142431020736694 50
5017
-1 26
5018
0 19
5019
0.9341365694999695 50
5020
0.9446338415145874 50
5021
0.9449027180671692 50
5022
-1 12
5023
0.9135857820510864 50
5024


KeyboardInterrupt: 

In [26]:
print(len(list_for_train_winning))

40


In [235]:
print(root.state[0] + root.state[1])
print(nextAction)
print(root.legal_moves[nextAction[0]])
print(env.tank_posx)
print(env.tank_posy)
print(env.state)

[[  0   0  10   1   5   1  20   0   1]
 [  0   0   0   1   2   1   0   0   0]
 [  1   1   0   1   1   1   0   0   0]
 [  1   0   1   0   1   0   0   0   1]
 [  1   1   1   1   1   1   1   1   1]
 [  1   1   0   0   1   1   0   0   0]
 [  0   0   1   1   1   1   0   0   1]
 [  0   0   0   0   2   1   0   1   1]
 [  0   0 -20   1   5   1 -10   0   1]]
[12, 16]
1
[[0 1]
 [8 8]]
[[0 6]
 [8 1]]
[[ 10   0   0   0   5   0   0   0   1]
 [  0   0   0   1   2   1  20   0   0]
 [  1   1   0   1   1   1   0   0   0]
 [  1   0   1   0   1   0   0   0   1]
 [  1   1   1   1   1   1   1   1   1]
 [  1   1   0   0   1   1   0   0   0]
 [  0   0   1   1   1   1   0   0   1]
 [  0   0   0   1   2   1   0   1   1]
 [  0 -20   0   0   5   1   0   1 -10]]


In [236]:
print(root.parent.parent.state[0] + root.parent.parent.state[1])
print(root.parent.state[0] + root.parent.state[1])
print(root.parent.number, root.number)

[[  0   0  10   1   5   1  20   0   1]
 [  0   0   0   1   2   1   0   0   0]
 [  1   1   0   1   1   1   0   0   0]
 [  1   0   1   0   1   0   0   0   1]
 [  1   1   1   1   1   1   1   1   1]
 [  1   1   0   0   1   1   0   0   0]
 [  0   0   1   1   1   1   0   0   1]
 [  0   0   0   0   2   0   1   1   1]
 [  0   0 -20   1   5   1 -10   0   1]]
[[  1   0  10   1   5   1  20   0   0]
 [  1   1   1   0   2   0   0   0   0]
 [  1   0   0   1   1   1   1   0   0]
 [  0   0   0   1   1   0   0   1   1]
 [  1   1   1   1   1   1   1   1   1]
 [  1   0   0   0   1   0   1   0   1]
 [  0   0   0   1   1   1   0   1   1]
 [  0   0   0   1   2   1   0   0   0]
 [  1   0 -20   1   5   1 -10   0   0]]
53 16


In [239]:
print(step_temp([12, 16], root.state.copy()))

[array([[[  0,   0,   0,   1,   5,   1,   0,   0,   1],
        [  0,   0,   0,   1,   2,   1,   0,   0,   0],
        [  1,   1,   0,   1,   1,   1,   0,   0,   0],
        [  1,   0,   1,   0,   1,   0,   0,   0,   1],
        [  1,   1,   1,   1,   1,   1,   1,   1,   1],
        [  1,   1,   0,   0,   1,   1,   0,   0,   0],
        [  0,   0,   1,   1,   1,   1,   0,   0,   1],
        [  0,   0,   0,   0,   2,   0,   1,   1,   1],
        [  0,   0,   0,   1,   5,   1,   0,   0,   1]],

       [[  0,   0,  10,   0,   0,   0,  20,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0],
        [  0,   0, -20,   0,   0,   0, -10,   0,   0]],

     

In [149]:
env.reset(0.75)
print(env.state)
print(model(cuda.FloatTensor(np.expand_dims(env.state_sum, axis = 0))))

[[  0   1  10   1   5   1  20   1   0]
 [  0   0   0   1   2   1   0   0   0]
 [  0   0   0   1   1   1   0   0   1]
 [  0   1   0   1   1   0   0   0   0]
 [  1   1   1   1   1   1   1   1   1]
 [  0   1   0   0   1   0   1   1   0]
 [  0   0   0   0   1   1   0   0   0]
 [  0   0   0   1   2   1   1   0   0]
 [  0   0 -20   1   5   1 -10   0   0]]
(tensor([[0.5000, 0.5000, 0.5000, 0.9998, 0.5000, 0.6460, 0.5000, 0.9998, 0.5000,
         0.5000, 0.6254, 0.5000, 0.9929, 0.5000, 0.5000, 0.9998, 0.5000, 0.5000,
         0.9998, 0.5000, 0.5000, 0.5000, 0.5000, 0.9998, 0.5000, 0.5000, 0.5000,
         0.5000, 0.9998, 0.5000, 0.6210, 0.5000, 0.5000, 0.5000, 0.9998, 0.9999,
         0.5000, 0.5000, 0.6404, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000,
         0.5000, 0.9998, 0.5000, 0.9998, 0.5000, 0.5000, 0.5000, 0.5000, 0.9998,
         0.5000, 0.5060, 0.5000, 0.5000, 0.5000, 0.5000, 0.9998, 0.9998, 0.5000,
         0.5000, 0.9999, 0.5000, 0.9998, 0.9998, 0.6288, 0.5000, 0.9998, 0.5000,

In [ ]:
node_temp = root
while node_temp.if_expanded:
    print(node_temp.state[0] + node_temp.state[1])
    for i in range(81):
        if node_temp.children[i] != None:
            node_temp = node_temp.children[i]
            break

In [22]:
torch.save(model, 'model4_5000.pkl')

E:\Anaconda3\Anaconda3\lib\site-packages\torch\serialization.py:241: UserWarning: Couldn't retrieve source code for container of type DRN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
E:\Anaconda3\Anaconda3\lib\site-packages\torch\serialization.py:241: UserWarning: Couldn't retrieve source code for container of type Flatten. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
